In [27]:
import os
import numpy as np
import pandas as pd

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
RAW_DATA_DIR = '../data/raw'
INTERIM_DIR = '../data/interim'

### Initial processing of dataset

In [5]:
title_akas = pd.read_csv('../data/raw/title_akas.tsv', sep='\t')
title_basics = pd.read_csv('../data/raw/title_basics.tsv', sep='\t')
title_crew = pd.read_csv('../data/raw/title_crew.tsv', sep='\t')
title_principals = pd.read_csv('../data/raw/title_principals.tsv', sep='\t')
title_ratings = pd.read_csv('../data/raw/title_ratings.tsv', sep='\t')
name_basics = pd.read_csv('../data/raw/name_basics.tsv', sep='\t')

Number of titles : 8718005
Number of ratings : 1217441
Number of actors : 11433516
Number of director and writer : 8711519
Number of principal cast/crew : 48987628


In [9]:
print("Number of titles    : {}".format(len(title_basics)))
print("Number of ratings   : {}".format(len(title_ratings)))
print("Number of actors    : {}".format(len(name_basics)))
print("Number of directors : {}".format(len(title_crew['directors'].unique())))
print("Number of writers   : {}".format(len(title_crew['writers'].unique())))

Number of titles    : 8718005
Number of ratings   : 1217441
Number of actors    : 11433516
Number of directors : 842767
Number of writers   : 1165618


In [11]:
# Listing all possible Types (we only need movie)

title_basics['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

#### Keep the movie titles that have a rating

In [30]:
# Keep only the titles that have a rating

titles = title_basics.merge(title_ratings, on='tconst')
titles

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1864
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.0,243
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1632
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",6.0,158
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2458
...,...,...,...,...,...,...,...,...,...,...,...
1217432,tt9916690,tvEpisode,Horrid Henry Delivers the Milk,Horrid Henry Delivers the Milk,0,2012,\N,10,"Adventure,Animation,Comedy",6.6,5
1217433,tt9916720,short,The Nun 2,The Nun 2,0,2019,\N,10,"Comedy,Horror,Mystery",5.5,187
1217434,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,\N,8.4,5
1217435,tt9916766,tvEpisode,Episode #10.15,Episode #10.15,0,2019,\N,43,"Family,Reality-TV",6.7,18


## Some Processes

- Keep 'movie' type only
- Convert the 'startYear' to integer values
- Drop the 'endYear' field as it is irrelevant for movies

In [22]:
def process_null_values(df):
    '''
    Change non standard Null value('\\N') to np.NaN so that pandas recognizes it
    
    params:
        - df (pd.DataFrame): Dataframe to process
    
    returns:
        - pd.DataFrame: Dataframe with np.NaN in place of Null
    '''
    return df.applymap(lambda x: np.NaN if (x=='\\N') else x)

In [31]:
# Taking care of NaN values
titles = process_null_values(titles)

# Drop 'endYear'
del titles['endYear']

# Filtering movies types
movies = titles[titles['titleType'].isin(['movie'])]

# Processing null values in startYear (average year)
mean_startYear = int(movies['startYear'].dropna().astype(int).mean())

movies['startYear'] = movies['startYear'].apply(lambda x: int(x) if (not pd.isna(x)) else mean_startYear)

# Resetting row numbers
movies.reset_index(drop=True, inplace=True)
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000502,movie,Bohemios,Bohemios,0,1905,100,NaN,4.5,14
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.1,753
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama,4.6,17
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,Drama,4.5,23
4,tt0000630,movie,Hamlet,Amleto,0,1908,NaN,Drama,3.8,24
...,...,...,...,...,...,...,...,...,...,...
275172,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,Thriller,5.8,1308
275173,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History",6.4,4425
275174,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,NaN,"Adventure,History,War",3.8,14
275175,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,123,Drama,8.3,6


#### Overview of columns having null values

In [32]:
movies.isnull().sum()

tconst                0
titleType             0
primaryTitle          0
originalTitle         0
isAdult               0
startYear             0
runtimeMinutes    27714
genres            10258
averageRating         0
numVotes              0
dtype: int64

#### Drop missing values for genres

In [38]:
# Drop data which has Null value in genres
movies = movies.loc[movies['genres'].notnull()]

#### Replacing Null values in runtimeMinutes by mean value

In [39]:
mean_runtime = int(movies['runtimeMinutes'].dropna().astype(int).mean())

movies['runtimeMinutes'] = movies['runtimeMinutes'].apply(lambda x: mean_runtime if (pd.isna(x)) else x)

In [52]:
movies.to_csv(os.path.join(INTERIM_DIR, 'movies.csv'))

### Processing title_crew, title_principals files

In [44]:
title_crew = process_null_values(title_crew)
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN
...,...,...,...
8711514,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8711515,tt9916850,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
8711516,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8711517,tt9916856,nm10538645,nm6951431


In [45]:
title_principals = process_null_values(title_principals)
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN
...,...,...,...,...,...,...
48987623,tt9916880,4,nm10535738,actress,NaN,"[""Horrid Henry""]"
48987624,tt9916880,5,nm0996406,director,principal director,NaN
48987625,tt9916880,6,nm1482639,writer,NaN,NaN
48987626,tt9916880,7,nm2586970,writer,books,NaN


#### Save only relevant data

In [46]:
# Filtering the crew and principals data from only relevant titles

# Add the titles we are considering to a set and then filter the dataset
considered_titles = set()
considered_titles.update(movies['tconst'].values)

title_principals = title_principals[title_principals['tconst'].isin(considered_titles)]
title_crew = title_crew[title_crew['tconst'].isin(considered_titles)]

print("Length of processed title_crew: {}".format(len(title_crew)))
print("Length of processed title_principals: {}".format(len(title_principals)))

Length of processed title_crew: 264919
Length of processed title_principals: 2375723


In [47]:
# Saving files as csvs to interim data folder

title_crew.to_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), index=False)
title_principals.to_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), index=False)

#### Filtering the names_basics dataframe for including only the names associated with considered movies for faster processing

**NOTE**: This filtering is only for saving time in prototyping further in the pipeline as I plan to calculate features for each person. In production, I would retain the entire name list and calculate the features on them as well in case a movie has a crew member who hasnt worked in a movie before (eg. an actor who only did TV series up until now).

**Observation:** IMDB lists the top billed actors ('star cast' or protagonist) first and the list decreases in order of importance

**Design decision:** From the cast of the movie, for genres computation, we only consider the **top 5 names** ('star cast' or actors playing most important characters) to reduce our calculations significantly. We already processed _title_principles_ to allow the same

In [42]:
names = process_null_values(name_basics)
names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0031983,tt0050419,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0038355,tt0117057,tt0037382"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0049189,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0080455,tt0078723,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0060827,tt0083922,tt0050986"
...,...,...,...,...,...,...
11433511,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
11433512,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
11433513,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
11433514,nm9993718,Aayush Nair,NaN,NaN,cinematographer,NaN


In [48]:
considered_name_ids = set()

def split_nameIDs(nconst_str):
    '''
    Split the nconst string by commas and filter out empty strings
    
    params:
        - nconst_str (str): String of comma separated nameIDs
    
    returns:
        - (list): List of nameIDs
    '''
    return [nameID for nameID in nconst_str.split(',') if nameID!='']

# Adding all directors
title_crew['directors'].dropna().apply(lambda x: considered_name_ids.update(split_nameIDs(x)))

# Adding all writers
title_crew['writers'].dropna().apply(lambda x: considered_name_ids.update(split_nameIDs(x)))

# Adding principal crew members
considered_name_ids.update(title_principals['nconst'].dropna().values)

print(len(considered_name_ids))

851540


In [49]:
names = names[names['nconst'].isin(considered_name_ids)]
names.reset_index(drop=True, inplace=True)
names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0031983,tt0050419,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0038355,tt0117057,tt0037382"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0049189,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0080455,tt0078723,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0060827,tt0083922,tt0050986"
...,...,...,...,...,...,...
851242,nm9993533,Lara Christie,NaN,NaN,actress,"tt0123527,tt0074965"
851243,nm9993594,Adeoluwa Owu,NaN,NaN,"cinematographer,director,camera_department","tt13204624,tt15892144,tt16423720,tt16253232"
851244,nm9993616,Ryan Mac Lennan,NaN,NaN,actor,tt4844148
851245,nm9993650,Marcin Balcerak,NaN,NaN,actor,tt8739208


In [51]:
names.to_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'))

#### We need all the ratings available because the plan is to calculate features for each person using the titles they are known for

In [54]:
title_ratings = process_null_values(title_ratings)
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1864
1,tt0000002,6.0,243
2,tt0000003,6.5,1632
3,tt0000004,6.0,158
4,tt0000005,6.2,2458
...,...,...,...
1217436,tt9916690,6.6,5
1217437,tt9916720,5.5,187
1217438,tt9916730,8.4,5
1217439,tt9916766,6.7,18


In [55]:
title_ratings.isnull().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [56]:
title_ratings.to_csv(os.path.join(INTERIM_DIR, 'title_ratings.csv'))

In [57]:
title_akas = process_null_values(title_akas)
title_akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0
...,...,...,...,...,...,...,...,...
31063819,tt9916852,5,Episódio #3.20,PT,pt,NaN,NaN,0
31063820,tt9916852,6,Episodio #3.20,IT,it,NaN,NaN,0
31063821,tt9916852,7,एपिसोड #3.20,IN,hi,NaN,NaN,0
31063822,tt9916856,1,The Wind,DE,NaN,imdbDisplay,NaN,0


In [58]:
title_akas.isnull().sum()

titleId                   0
ordering                  0
title                     4
region              1906958
language            6089260
types              25844902
attributes         30827344
isOriginalTitle        2187
dtype: int64

In [59]:
# Filter the dataframe to include titles we consider and no more
title_akas = title_akas[title_akas['titleId'].isin(considered_titles)][['titleId', 'ordering']]
title_akas

,titleId,ordering
2183,tt0000574,1
2184,tt0000574,2
2185,tt0000574,3
2186,tt0000574,4
2187,tt0000574,5
...,...,...
31063155,tt9916428,2
31063156,tt9916428,3
31063157,tt9916428,4
31063158,tt9916428,5


In [60]:
title_akas.to_csv(os.path.join(INTERIM_DIR, 'title.akas.csv'))